# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'10-04-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'10-04-2020'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-10-05 04:23:33,33.93911,67.709953,39341,1462,32852,5027.0,Afghanistan,101.060048,3.716225
1,NaN,NaN,NaN,Albania,2020-10-05 04:23:33,41.15330,20.168300,14266,396,8675,5195.0,Albania,495.725902,2.775831
2,NaN,NaN,NaN,Algeria,2020-10-05 04:23:33,28.03390,1.659600,52136,1760,36578,13798.0,Algeria,118.893409,3.375786
3,NaN,NaN,NaN,Andorra,2020-10-05 04:23:33,42.50630,1.521800,2110,53,1540,517.0,Andorra,2730.861321,2.511848
4,NaN,NaN,NaN,Angola,2020-10-05 04:23:33,-11.20270,17.873900,5402,195,2577,2630.0,Angola,16.436305,3.609774


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,9/25/20,9/26/20,9/27/20,9/28/20,9/29/20,9/30/20,10/1/20,10/2/20,10/3/20,10/4/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,39186,39192,39227,39233,39254,39268,39285,39290,39297,39341
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,13045,13153,13259,13391,13518,13649,13806,13965,14117,14266
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,50754,50914,51067,51213,51368,51530,51690,51847,51995,52136


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,9/25/20,9/26/20,9/27/20,9/28/20,9/29/20,9/30/20,10/1/20,10/2/20,10/3/20,10/4/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1451,1453,1453,1455,1458,1458,1458,1458,1462,1462
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,373,375,377,380,384,387,388,389,392,396
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,1707,1711,1714,1719,1726,1736,1741,1749,1756,1760


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,9/25/20,9/26/20,9/27/20,9/28/20,9/29/20,9/30/20,10/1/20,10/2/20,10/3/20,10/4/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,32619,32635,32642,32642,32746,32789,32842,32842,32842,32852
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,7309,7397,7397,7629,7732,7847,8077,8342,8536,8675
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,35654,35756,35860,35962,36063,36174,36282,36385,36482,36578


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,9/25/20,9/26/20,9/27/20,9/28/20,9/29/20,9/30/20,10/1/20,10/2/20,10/3/20,10/4/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1757,1764,1773,1785,1787,1791,1798,1805,1818,1828
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,5456,5477,5526,5588,5606,5640,5997,6024,6048,6073
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,873,882,885,886,886,896,898,902,921,921


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,9/25/20,9/26/20,9/27/20,9/28/20,9/29/20,9/30/20,10/1/20,10/2/20,10/3/20,10/4/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,25,25,25,25,27,27,28,27,27,27
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,50,50,50,50,50,52,53,53,53,53
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,7,7,7,7,7,7,7,7,7,7


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
632,1001.0,Autauga,Alabama,US,2020-10-05 04:23:33,32.539527,-86.644082,1828,27,0,1801.0,"Autauga, Alabama, US",3271.939716,1.477024
669,1075.0,Lamar,Alabama,US,2020-10-05 04:23:33,33.779950,-88.096680,400,2,0,398.0,"Lamar, Alabama, US",2897.500905,0.500000
670,1077.0,Lauderdale,Alabama,US,2020-10-05 04:23:33,34.901719,-87.656247,1815,37,0,1778.0,"Lauderdale, Alabama, US",1957.316481,2.038567


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,7420971,209794,2911699
India,6549373,101782,5509966
Brazil,4915289,146352,4375354


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,7420971,209794,2911699,4299478,2020-10-05 04:23:33,37.923092,-91.345036
India,6549373,101782,5509966,937625,2020-10-05 04:23:33,23.088275,81.806127
Brazil,4915289,146352,4375354,393583,2020-10-05 04:23:33,-12.669522,-48.480493
Russia,1209039,21260,975488,212291,2020-10-05 04:23:33,54.546312,62.120860
Colombia,855052,26712,761674,66666,2020-10-05 04:23:33,5.349139,-74.190797


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,832713,16147,0
Texas,790194,16320,0
Florida,716459,14671,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,832713,16147,0,816566,2020-10-05 04:23:33,37.843962,-120.728594
Texas,790194,16320,0,773874,2020-10-05 04:23:33,31.660643,-98.653069
Florida,716459,14671,0,701788,2020-10-05 04:23:33,28.940755,-82.700744
New York,464582,33205,0,431377,2020-10-05 04:23:33,42.544151,-75.474183
Georgia,322925,7162,0,315763,2020-10-05 04:23:33,32.808532,-83.577430


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,274565,6647,0
Florida,Miami-Dade,172205,3334,0
Illinois,Cook,148381,5256,0
Texas,Harris,147808,2626,0
Arizona,Maricopa,143183,3426,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,274565,6647,0,267918,2020-10-05 04:23:33,34.308284,-118.228241,6037.0
Florida,Miami-Dade,172205,3334,0,168871,2020-10-05 04:23:33,25.611236,-80.551706,12086.0
Illinois,Cook,148381,5256,0,143125,2020-10-05 04:23:33,41.841448,-87.816588,17031.0
Texas,Harris,147808,2626,0,145182,2020-10-05 04:23:33,29.858649,-95.393395,48201.0
Arizona,Maricopa,143183,3426,0,139757,2020-10-05 04:23:33,33.348359,-112.491815,4013.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-02,39290,13965,51847,2110,5211,106,779689,51382,27121,46374,...,469769,2097,57454,76820,1096,40766,10,2040,14830,7858
2020-10-03,39297,14117,51995,2110,5370,107,790818,51925,27135,47432,...,482654,2122,58238,77646,1096,41078,10,2041,14974,7885
2020-10-04,39341,14266,52136,2110,5402,107,798486,52496,27148,48146,...,505619,2145,58612,78434,1096,41498,10,2041,15052,7888


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-02,1458,389,1749,53,189,3,20599,966,893,803,...,42358,48,472,643,35,321,1,589,333,228
2020-10-03,1462,392,1756,53,193,3,20795,972,894,809,...,42407,48,477,649,35,329,1,589,333,228
2020-10-04,1462,396,1760,53,195,3,21018,977,894,813,...,42440,48,480,653,35,330,1,591,333,228


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-02,32842,8342,36385,1540,2215,94,614515,44406,24848,37186,...,2389,1824,53955,67216,1020,32944,8,1307,13980,6322
2020-10-03,32842,8536,36482,1540,2436,96,626114,44583,24864,38045,...,2396,1831,54854,68098,1020,33442,8,1320,14094,6327
2020-10-04,32852,8675,36578,1540,2577,96,636672,44672,24888,38629,...,2403,1844,55281,68917,1020,34698,8,1320,14187,6359


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,9/25/20,9/26/20,9/27/20,9/28/20,9/29/20,9/30/20,10/1/20,10/2/20,10/3/20,10/4/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1757,1764,1773,1785,1787,1791,1798,1805,1818,1828
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,366,367,370,377,382,382,386,393,397,400
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,1698,1713,1723,1740,1751,1759,1772,1787,1807,1815
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,616,620,621,628,634,636,644,647,652,652
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,5887,5897,5927,5945,5971,6017,6043,6068,6117,6141


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-10-02,156698,8073,0,219763,85779,826013,71876,58297,20937,49,...,198403,782306,75157,1768,1326,149490,88810,16307,127906,6214
2020-10-03,158380,8217,0,220399,86525,829521,72533,58297,21125,49,...,199595,787425,76225,1778,1327,150583,89419,16481,130798,6365
2020-10-04,159169,8405,0,220754,87013,832713,73054,58297,21243,49,...,201210,790194,77618,1785,1327,151642,89874,16635,132663,6504


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-10-02        1805    6024     902  675   1642     612    922    3628   
2020-10-03        1818    6048     921  678   1655     613    924    3660   
2020-10-04        1828    6073     921  686   1656     613    926    3679   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-10-02         1185      631  ...     259     41      309      121   
2020-10-03         1204      639  ...     260     41      318      122   
2020-10-04         1205      639  ...     265     44      324      123   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-10-02            345   597   358          0      117     39  
2020-10-03            357   602   359          0      118     41  
2020-10-04            361   616   360          0      119     42  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-10-02,2550,57,0,5693,1391,16054,2057,4513,642,0,...,2515,16224,474,58,20,3247,2143,361,1353,53
2020-10-03,2558,58,0,5705,1407,16115,2060,4513,645,0,...,2560,16295,476,58,20,3267,2142,363,1372,53
2020-10-04,2558,58,0,5706,1425,16147,2068,4513,645,0,...,2577,16320,478,58,20,3270,2142,364,1377,53


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-10-02          27      53       7   10     15      15     40      44   
2020-10-03          27      53       7   10     15      15     40      44   
2020-10-04          27      53       7   10     15      15     40      44   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-10-02           42       13  ...       2      1        4        1   
2020-10-03           42       13  ...       2      1        4        1   
2020-10-04           42       13  ...       2      1        4        1   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-10-02              2     1     2          0        6      0  
2020-10-03              2     1     2          0        6      0  
2020-10-04              2     1     2          0        6      0  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-02,0.000127,0.011517,0.003037,0.029268,0.018968,0.049505,0.019199,0.010462,0.000443,0.015059,...,0.015113,0.017467,0.004616,0.010404,0.000913,0.011011,0.0,0.00049,0.001892,0.001019
2020-10-03,0.000178,0.010884,0.002855,0.000000,0.030512,0.009434,0.014274,0.010568,0.000516,0.022815,...,0.027428,0.011922,0.013646,0.010752,0.000000,0.007653,0.0,0.00049,0.009710,0.003436
2020-10-04,0.001120,0.010555,0.002712,0.000000,0.005959,0.000000,0.009696,0.010997,0.000479,0.015053,...,0.047581,0.010839,0.006422,0.010149,0.000000,0.010224,0.0,0.00000,0.005209,0.000380


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-02,0.000000,0.002577,0.004595,0.0,0.021622,0.0,0.015329,0.003115,0.003371,0.001247,...,0.001561,0.0,0.002123,0.012598,0.0,0.009434,0.0,0.003407,0.0,0.0
2020-10-03,0.002743,0.007712,0.004002,0.0,0.021164,0.0,0.009515,0.006211,0.001120,0.007472,...,0.001157,0.0,0.010593,0.009331,0.0,0.024922,0.0,0.000000,0.0,0.0
2020-10-04,0.000000,0.010204,0.002278,0.0,0.010363,0.0,0.010724,0.005144,0.000000,0.004944,...,0.000778,0.0,0.006289,0.006163,0.0,0.003040,0.0,0.003396,0.0,0.0


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-02,0.000000,0.032809,0.002839,0.075419,0.063881,0.021739,0.018860,0.004229,0.002501,0.019465,...,0.002518,0.008292,0.002229,0.014658,0.009901,0.011266,0.0,0.007710,0.001361,0.001584
2020-10-03,0.000000,0.023256,0.002666,0.000000,0.099774,0.021277,0.018875,0.003986,0.000644,0.023100,...,0.002930,0.003838,0.016662,0.013122,0.000000,0.015117,0.0,0.009946,0.008155,0.000791
2020-10-04,0.000304,0.016284,0.002631,0.000000,0.057882,0.000000,0.016863,0.001996,0.000965,0.015350,...,0.002922,0.007100,0.007784,0.012027,0.000000,0.037558,0.0,0.000000,0.006599,0.005058


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-10-02,0.006125,0.015727,NaN,0.002514,0.011294,0.004631,0.009537,0.009612,0.007216,0.0,...,0.004918,0.007171,0.014949,0.007407,0.000000,0.006423,0.007876,0.017534,0.021932,0.021535
2020-10-03,0.010734,0.017837,NaN,0.002894,0.008697,0.004247,0.009141,0.000000,0.008979,0.0,...,0.006008,0.006543,0.014210,0.005656,0.000754,0.007312,0.006857,0.010670,0.022610,0.024300
2020-10-04,0.004982,0.022879,NaN,0.001611,0.005640,0.003848,0.007183,0.000000,0.005586,0.0,...,0.008091,0.003517,0.018275,0.003937,0.000000,0.007033,0.005088,0.009344,0.014259,0.021838


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-10-02      0.003893  0.004502  0.004454  0.004464  0.004896  0.000000   
2020-10-03      0.007202  0.003984  0.021064  0.004444  0.007917  0.001634   
2020-10-04      0.005501  0.004134  0.000000  0.011799  0.000604  0.000000   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-10-02      0.003264  0.011430  0.003387  0.011218  ...  0.027778   
2020-10-03      0.002169  0.008820  0.016034  0.012678  ...  0.003861   
2020-10-04      0.002165  0.005191  0.000831  0.000000  ...  0.019231   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-10-02      0.078947  0.016447  0.034188   0.008772  0.010152  0.005618   
2020-10-03      0.000000  0.029126  0.008264   0.034783  0.008375  0.002793   
2020-10-04      0.073171  0.018868  0.008197   0.011204  0.023256  0.002786   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-10-02            NaN  0.008621  0.000000  
2020-10-03            NaN  0.008547  0.051282  
2020-10-04            NaN  0.008475  0.024390  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-10-02,0.000785,0.000000,NaN,0.003349,0.005058,0.005071,0.001461,0.000443,0.009434,NaN,...,0.005598,0.005828,0.032680,0.0,0.0,0.006822,0.005159,0.005571,0.003709,0.0
2020-10-03,0.003137,0.017544,NaN,0.002108,0.011503,0.003800,0.001458,0.000000,0.004673,NaN,...,0.017893,0.004376,0.004219,0.0,0.0,0.006160,-0.000467,0.005540,0.014043,0.0
2020-10-04,0.000000,0.000000,NaN,0.000175,0.012793,0.001986,0.003883,0.000000,0.000000,NaN,...,0.006641,0.001534,0.004202,0.0,0.0,0.000918,0.000000,0.002755,0.003644,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State   Alabama                                                     \
Admin2           Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-10-02     -0.035714     0.0     0.0  0.0    0.0 -0.0625    0.0     0.0   
2020-10-03      0.000000     0.0     0.0  0.0    0.0  0.0000    0.0     0.0   
2020-10-04      0.000000     0.0     0.0  0.0    0.0  0.0000    0.0     0.0   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-10-02          0.0      0.0  ...     0.0    0.0      0.0      0.0   
2020-10-03          0.0      0.0  ...     0.0    0.0      0.0      0.0   
2020-10-04          0.0      0.0  ...     0.0    0.0      0.0      0.0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-10-02            0.0   0.0   0.0        NaN      0.0    NaN  
2020-10-03            0.0   0.0   0.0        NaN      0.0    NaN  
2020-10-04            0.0   0.0   0.0        NaN      0.0    NaN  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-02,0.000275,0.011018,0.003067,0.022435,0.020762,0.025282,0.018859,0.009566,0.000504,0.016392,...,0.015081,0.012353,0.006391,0.010904,0.001833,0.010632,1.095640e-32,0.001065,0.002432,0.001155
2020-10-03,0.000227,0.010951,0.002961,0.011217,0.025637,0.017358,0.016566,0.010067,0.000510,0.019603,...,0.021255,0.012138,0.010018,0.010828,0.000917,0.009143,5.478201e-33,0.000777,0.006071,0.002295
2020-10-04,0.000673,0.010753,0.002836,0.005609,0.015798,0.008679,0.013131,0.010532,0.000495,0.017328,...,0.034418,0.011488,0.008220,0.010489,0.000458,0.009684,2.739100e-33,0.000389,0.005640,0.001338


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-02,0.000191,0.003991,0.004158,4.270089e-18,0.018416,3.367057e-52,0.063196,0.003073,0.003166,0.002678,...,0.001476,0.001351,0.003929,0.012019,3.703896e-11,0.015333,0.0,0.001711,0.000757,0.000144
2020-10-03,0.001467,0.005852,0.004080,2.135044e-18,0.019790,1.683529e-52,0.036355,0.004642,0.002143,0.005075,...,0.001316,0.000676,0.007261,0.010675,1.851948e-11,0.020128,0.0,0.000855,0.000379,0.000072
2020-10-04,0.000734,0.008028,0.003179,1.067522e-18,0.015076,8.417643e-53,0.023540,0.004893,0.001071,0.005010,...,0.001047,0.000338,0.006775,0.008419,9.259740e-12,0.011584,0.0,0.002125,0.000189,0.000036


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-02,0.000776,0.027595,0.002908,0.054470,0.061358,0.010870,0.017020,0.004164,0.001764,0.020765,...,0.003391,0.009375,0.005636,0.014999,0.005896,0.019018,6.419766e-35,0.007653,0.001643,0.005122
2020-10-03,0.000388,0.025425,0.002787,0.027235,0.080566,0.016073,0.017948,0.004075,0.001204,0.021933,...,0.003160,0.006606,0.011149,0.014061,0.002948,0.017067,3.209883e-35,0.008800,0.004899,0.002956
2020-10-04,0.000346,0.020855,0.002709,0.013618,0.069224,0.008037,0.017405,0.003036,0.001084,0.018641,...,0.003041,0.006853,0.009467,0.013044,0.001474,0.027312,1.604942e-35,0.004400,0.005749,0.004007


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-10-02,0.006263,0.015474,NaN,0.002546,0.011541,0.004245,0.009038,0.006650,0.007007,2.118473e-61,...,0.005783,0.006496,0.013966,0.004627,0.001700,0.005253,0.006953,0.014252,0.021679,0.021294
2020-10-03,0.008498,0.016656,NaN,0.002720,0.010119,0.004246,0.009090,0.003325,0.007993,1.059237e-61,...,0.005896,0.006520,0.014088,0.005141,0.001227,0.006282,0.006905,0.012461,0.022145,0.022797
2020-10-04,0.006740,0.019768,NaN,0.002165,0.007880,0.004047,0.008136,0.001662,0.006789,5.296183e-62,...,0.006993,0.005018,0.016181,0.004539,0.000614,0.006657,0.005997,0.010903,0.018202,0.022318


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-10-02      0.003675  0.019794  0.004466  0.006679  0.004106  0.001627   
2020-10-03      0.005439  0.011889  0.012765  0.005562  0.006012  0.001630   
2020-10-04      0.005470  0.008011  0.006383  0.008681  0.003308  0.000815   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-10-02      0.002418  0.007904  0.002941  0.008538  ...  0.025170   
2020-10-03      0.002294  0.008362  0.009487  0.010608  ...  0.014515   
2020-10-04      0.002229  0.006777  0.005159  0.005304  ...  0.016873   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-10-02      0.088562  0.017391  0.035001   0.008206  0.015954  0.003537   
2020-10-03      0.044281  0.023259  0.021633   0.021494  0.012165  0.003165   
2020-10-04      0.058726  0.021063  0.014915   0.016349  0.017710  0.002975   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-10-02      -0.571428  0.005026  0.010056  
2020-10-03      -0.571428  0.006786  0.030669  
2020-10-04      -0.571428  0.007631  0.027530  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-10-02,0.002761,0.006220,NaN,0.003321,0.009029,0.005500,0.001549,0.000516,0.005101,NaN,...,0.010518,0.006231,0.017654,6.135151e-20,0.001645,0.006338,0.004166,0.009405,0.009374,0.015021
2020-10-03,0.002949,0.011882,NaN,0.002714,0.010266,0.004650,0.001504,0.000258,0.004887,NaN,...,0.014206,0.005304,0.010937,3.067575e-20,0.000822,0.006249,0.001850,0.007472,0.011708,0.007511
2020-10-04,0.001475,0.005941,NaN,0.001445,0.011529,0.003318,0.002694,0.000129,0.002443,NaN,...,0.010423,0.003419,0.007569,1.533788e-20,0.000411,0.003583,0.000925,0.005114,0.007676,0.003755


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                              \
Admin2           Autauga   Baldwin       Barbour      Bibb    Blount   
2020-10-02     -0.003577  0.009850  4.759163e-15 -0.010220  0.000072   
2020-10-03     -0.001789  0.004925  2.379581e-15 -0.005110  0.000036   
2020-10-04     -0.000894  0.002462  1.189791e-15 -0.002555  0.000018   

Province_State                                                        ...  \
Admin2           Bullock    Butler   Calhoun      Chambers  Cherokee  ...   
2020-10-02     -0.005654  0.001628  0.000531  3.815179e-07 -0.000578  ...   
2020-10-03     -0.002827  0.000814  0.000265  1.907589e-07 -0.000289  ...   
2020-10-04     -0.001414  0.000407  0.000133  9.537947e-08 -0.000145  ...   

Province_State   Wyoming                                                   \
Admin2              Park Platte  Sheridan Sublette Sweetwater Teton Uinta   
2020-10-02      0.000015    0.0  0.000023      0.0        0.0   0.0   0.0   
2020-10-03      0.000008    0.0  0.000011      0.0        0.0   0.0   0.0   
2020-10-04      0.000004    0.0  0.000006      0.0        0.0   0.0   0.0   

Province_State                             
Admin2         Unassigned Washakie Weston  
2020-10-02      -0.999969      0.0    NaN  
2020-10-03      -0.999969      0.0    NaN  
2020-10-04      -0.999969      0.0    NaN  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,negative,pending,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201004,AK,9376,467442,NaN,476818,51.0,NaN,NaN,NaN,...,476818,0,0,d10acf299a4fff49e734775bc42aeaebfbe9776a,0,0,0,0,0,NaN
1,20201004,AL,159169,1022073,NaN,1162518,757.0,17420.0,NaN,1845.0,...,1181242,0,0,8b4f0135c1c94f579c47b8ec993ab4f2ed59114c,0,0,0,0,0,NaN
2,20201004,AR,87013,989037,NaN,1072343,499.0,5550.0,228.0,NaN,...,1076050,18,23,34347bac3d8efea7c2f887c475cf58798d4e87af,0,0,0,0,0,NaN
3,20201004,AS,0,1616,NaN,1616,NaN,NaN,NaN,NaN,...,1616,0,0,1f524de116bdc1d7b5dbfe1bd5cb055163994bb1,0,0,0,0,0,NaN
4,20201004,AZ,220754,1280104,NaN,1496131,545.0,22294.0,133.0,NaN,...,1500858,1,14,3320c539367d00aec197dc01914ddeaa41fb68de,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,negative,pending,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201004,AK,9376.0,467442.0,NaN,476818.0,51.0,NaN,NaN,NaN,...,476818,0,0,d10acf299a4fff49e734775bc42aeaebfbe9776a,0,0,0,0,0,NaN
1,20201004,AL,159169.0,1022073.0,NaN,1162518.0,757.0,17420.0,NaN,1845.0,...,1181242,0,0,8b4f0135c1c94f579c47b8ec993ab4f2ed59114c,0,0,0,0,0,NaN
2,20201004,AR,87013.0,989037.0,NaN,1072343.0,499.0,5550.0,228.0,NaN,...,1076050,18,23,34347bac3d8efea7c2f887c475cf58798d4e87af,0,0,0,0,0,NaN
3,20201004,AS,0.0,1616.0,NaN,1616.0,NaN,NaN,NaN,NaN,...,1616,0,0,1f524de116bdc1d7b5dbfe1bd5cb055163994bb1,0,0,0,0,0,NaN
4,20201004,AZ,220754.0,1280104.0,NaN,1496131.0,545.0,22294.0,133.0,NaN,...,1500858,1,14,3320c539367d00aec197dc01914ddeaa41fb68de,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,negative,pending,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-10-04,AK,9376,467442,NaN,476818,51.0,NaN,NaN,NaN,6.0,...,476818,0,0,d10acf299a4fff49e734775bc42aeaebfbe9776a,0,0,0,0,0,NaN
2020-10-04,AL,159169,1022073,NaN,1162518,757.0,17420.0,NaN,1845.0,NaN,...,1181242,0,0,8b4f0135c1c94f579c47b8ec993ab4f2ed59114c,0,0,0,0,0,NaN
2020-10-04,AR,87013,989037,NaN,1072343,499.0,5550.0,228.0,NaN,88.0,...,1076050,18,23,34347bac3d8efea7c2f887c475cf58798d4e87af,0,0,0,0,0,NaN
2020-10-04,AS,0,1616,NaN,1616,NaN,NaN,NaN,NaN,NaN,...,1616,0,0,1f524de116bdc1d7b5dbfe1bd5cb055163994bb1,0,0,0,0,0,NaN
2020-10-04,AZ,220754,1280104,NaN,1496131,545.0,22294.0,133.0,NaN,59.0,...,1500858,1,14,3320c539367d00aec197dc01914ddeaa41fb68de,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,negative,pending,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-10-04,AK,9376.0,467442.0,NaN,476818.0,51.0,NaN,NaN,NaN,6.0,...,476818,0,0,d10acf299a4fff49e734775bc42aeaebfbe9776a,0,0,0,0,0,NaN
2020-10-04,AL,159169.0,1022073.0,NaN,1162518.0,757.0,17420.0,NaN,1845.0,NaN,...,1181242,0,0,8b4f0135c1c94f579c47b8ec993ab4f2ed59114c,0,0,0,0,0,NaN
2020-10-04,AR,87013.0,989037.0,NaN,1072343.0,499.0,5550.0,228.0,NaN,88.0,...,1076050,18,23,34347bac3d8efea7c2f887c475cf58798d4e87af,0,0,0,0,0,NaN
2020-10-04,AS,0.0,1616.0,NaN,1616.0,NaN,NaN,NaN,NaN,NaN,...,1616,0,0,1f524de116bdc1d7b5dbfe1bd5cb055163994bb1,0,0,0,0,0,NaN
2020-10-04,AZ,220754.0,1280104.0,NaN,1496131.0,545.0,22294.0,133.0,NaN,59.0,...,1500858,1,14,3320c539367d00aec197dc01914ddeaa41fb68de,0,0,0,0,0,NaN
